In [19]:
import json
import os
import torch
from llama_index.core import Settings, VectorStoreIndex, SummaryIndex, SimpleDirectoryReader, PromptTemplate
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Paths
secrets_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\user_files\secrets.json"
docs_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs"

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Using GPU: {gpu_name}")
else:
    print("CUDA is not available. Using CPU.")

# Load secrets
with open(secrets_path, 'r') as file:
    data = json.load(file)

OPENAI_API_KEY = data["OPENAI_KEY"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
system_prompt_gpt4 = """
answer questions from a job applicant's perspective
"""
# Initialize necessary language models
gpt3_5_llm = OpenAI(model="gpt-3.5-turbo")
gpt3_5_embed = OpenAIEmbedding(model="text-embedding-ada-002", system_prompt = system_prompt_gpt4)
# System prompt for the LLM



# Load documents
documents = SimpleDirectoryReader(docs_path).load_data()


# Configure LLM settings
Settings.llm = gpt3_5_llm
Settings.embed_model = gpt3_5_embed

# Create an index from documents
summary_index = VectorStoreIndex(documents)

# Create a query engine
query_engine_summary = summary_index.as_query_engine(response_mode="refine")

# Define the prompt template
qa_prompt_tmpl_str_gpt4 = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,
it is very important that you follow the instructions clearly and answer based on the given information.
Answer the query in the format requested in the query.
Do not leave blanks in the answers.
Always answer the questions in the form of a cover letter and in first person.
If asked for a cover letter, write a short cover letter talking about your previous work experience and how it would make you a good fit for the given role.
If asked about why you want to work at a certain company, write a concise cover letter including the company's name and talking about your previous work experience and how it would make you a good fit for the given role.

Query: {query_str}
Answer: \
"""

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str_gpt4)

# Update query engine prompts
query_engine_summary.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Example query
response = query_engine_summary.query("why do you want to work at Microsoft")
print(response.response)


Using GPU: NVIDIA GeForce GTX 1650
I am excited about the opportunity to work at Microsoft because of the company's reputation for innovation and cutting-edge technology. With my background in data science, computer science, and experience as a Data Analyst and Data Engineer, I believe I can contribute effectively to Microsoft's mission of empowering individuals and organizations to achieve more through technology.

My experience in utilizing data to drive insights and make informed decisions aligns well with Microsoft's focus on leveraging data to create impactful solutions. I am particularly interested in the opportunity to work with cloud computing technologies like Azure, given my experience with AWS and Azure.

I am confident that my skills and experience make me a strong fit for a role at Microsoft, and I am eager to contribute to the company's success.

Thank you for considering my application.

Sincerely,
Ganesh Raj Kyatham
